In [2]:
import pandas as pd
info = pd.read_csv("WiDS Datathon 2020 Dictionary.csv")

# Dropping Unimportant Columns

In [ ]:
drop_cols = ['encounter_id','patient_id','icu_id', 'hospital_id', 'readmission_status']
drop_cols_test = ['encounter_id','patient_id','icu_id','hospital_death','hospital_id', 'readmission_status']

train = pd.read_csv("training_v2.csv").drop(drop_cols, axis=1)
test = pd.read_csv("unlabeled.csv").drop(drop_cols_test, axis=1)

In [ ]:
target = "hospital_death"

In [ ]:
print(f'Rows in train data : {train.shape[0]} and columns in train data: {train.shape[1]}')
print(f'Rows in test data  : {test.shape[0]} and columns in train data: {test.shape[1]}')

# Data Cleaning - Fill in NA/empty data

In [ ]:
train['age'].fillna(train['age'].mean(), inplace = True)
train['bmi'].fillna(train['bmi'].median(), inplace = True)
train['ethnicity'].fillna(train['ethnicity'].mode()[0], inplace = True)
train['weight'].fillna(train['weight'].median(), inplace = True)
train['height'].fillna(train['height'].median(), inplace = True)
train['gender'].fillna(train['gender'].mode()[0], inplace = True)

In [ ]:
for i in range(41,169,1):
    train.iloc[:,i] = train.iloc[:,i].fillna(0)

In [ ]:
cols2 = train.isna().sum().reset_index()["index"]
val2 = train.isna().sum().reset_index()[0]

In [ ]:
for c, v in zip(cols2, val2):
    print(c, v)

In [ ]:
test['age'].fillna(test['age'].mean(), inplace = True)
test['bmi'].fillna(test['bmi'].median(), inplace = True)
test['ethnicity'].fillna(test['ethnicity'].mode()[0], inplace = True)
test['weight'].fillna(test['weight'].median(), inplace = True)
test['height'].fillna(test['height'].median(), inplace = True)
test['gender'].fillna(test['gender'].mode()[0], inplace = True)

In [ ]:
for i in range(41,169,1):
    test.iloc[:,i] = test.iloc[:,i].fillna(0)

In [ ]:
cat_cols = ['gcs_verbal_apache',
     'icu_admit_source',
     'icu_type',
     'icu_stay_type',
     'ethnicity',
     'gender',
     'apache_2_bodysystem',
     'gcs_eyes_apache',
     'gcs_motor_apache',
     'apache_3j_bodysystem']

In [51]:
for col in cat_cols:
    train[col] = pd.get_dummies(train[[col]])
    test[col] = pd.get_dummies(test[[col]])

# Training Model

In [52]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier

In [53]:
train = train.dropna()
y = train[target]
X = train.drop(['hospital_admit_source'], axis=1)
X = X.drop(target, axis=1)

X_train, valid_features, Y_train, valid_y = train_test_split(X, y, test_size = 0.25, random_state = 1)

In [54]:
gradient_boost = GradientBoostingClassifier(random_state=1)
random_forest = RandomForestClassifier(n_estimators=100)

voting = VotingClassifier(estimators=[('rfc', random_forest),('gbc', gradient_boost)], voting='soft')
voting = voting.fit(X_train, Y_train)
predict_y = voting.predict(valid_features)

In [55]:
test = test.dropna()
test = test.drop(['hospital_admit_source'], axis=1)
test_copy = test.copy()
test_copy["hospital_death"] = voting.predict(test)

# test_copy[["encounter_id","hospital_death"]].to_csv("submission1.csv",index=False)
# test_copy[["encounter_id","hospital_death"]].head()

In [1]:
test_copy['hospital_death'].to_csv("submission1.csv", index=True, index_label='encounter_id')

NameError: name 'test_copy' is not defined